In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.max_rows', None)     
pd.set_option('display.float_format', '{:.4f}'.format)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler

In [10]:
player_stats = pd.read_csv('../data/player_stats.csv')
match_events = pd.read_csv('../data/match_event_base.csv')
shots_events = pd.read_csv('../data/shot_events.csv')
card_events = pd.read_csv('../data/card_events.csv')
match_stats = pd.read_csv('../data/norm_matches.csv')
rounds = pd.read_csv('../data/matches.csv')
players = pd.read_csv('../data/players.csv', encoding='utf-8')
seasons = pd.read_csv('../data/seasons.csv')


#Agregar globales de cada jugador con su equipo, porcentaje de key_passes de el vs todo el team, etc para todas las variables descriptivas del rol.

In [33]:
player_stats.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,ground_duels_total,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal
0,1,11352250,84844,60,f,G,1,1900000.0000,6.4000,90,46,0,0,0,27,39,0,6,18,0,0,0,0,0,0,0,0,12,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,-0.2972,0,0,0,0.0000,0.0053,0,0,0,0,0,0
1,2,11352250,943645,60,f,D,37,11600000.0000,6.9000,75,52,0,0,0,32,36,1,1,1,0,1,0,0,0,0,0,0,6,0,6,6,0,3,0,3,2,0,0,0,1,1,2,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0691,0,0,0,0,0,0
2,3,11352250,1023567,60,f,D,27,37000000.0000,6.5000,90,102,0,0,0,77,96,0,4,15,0,0,0,0,0,0,0,0,20,0,5,8,4,5,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0122,0,0,0,0,0,0
3,4,11352250,830659,60,f,D,25,22000000.0000,6.7000,60,62,0,0,0,43,51,0,3,8,0,0,0,0,0,0,0,0,10,1,3,6,1,2,2,0,2,1,3,1,2,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0172,0,0,0,0,0,0
4,5,11352250,1097425,60,f,D,3,36000000.0000,6.8000,90,75,0,0,0,31,40,1,2,3,1,4,0,0,0,0,0,0,17,1,10,6,4,2,6,3,2,0,3,0,1,0,3,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0493,0,0,0,0,0,0


In [18]:
match_stats.head()

,team_match_stat_id,match_id,team_id,opp_team_id,home_away,period,formation,average_team_rating,total_team_market_value_eur,possession_percentage,big_chances,total_shots,saves,corners,fouls,passes_successful,passes_total,passes_percentage,tackles_successful,tackles_total,tackles_won_percentage,free_kicks,yellow_cards,red_cards,shots_on_target,hit_woodwork,shots_off_target,blocked_shots,shots_inside_box,shots_outside_box,big_chances_missed,fouled_final_third,offsides,accurate_passes_percentage,throw_ins,final_third_entries,long_balls_successful,long_balls_total,long_balls_percentage,crosses_successful,crosses_total,crosses_percentage,duels_won_successful,duels_won_total,duels_won_percentage,dispossessed,ground_duels_successful,ground_duels_total,ground_duels_percentage,aerial_duels_successful,aerial_duels_total,aerial_duels_percentage,dribbles_successful,dribbles_total,dribbles_percentage,interceptions,clearances,goal_kicks,expected_goals,touches_in_penalty_area,passes_in_final_third,recoveries,errors_lead_to_shot,goals_prevented,big_saves,errors_lead_to_goal,penalty_saves,big_chances_scored,round_number,season_id,match_datetime_utc
0,1,11352250,60,37,H,ALL,4-2-3-1,6.7900,300785000.0000,0.6300,1,14,1,10,9,0,517,NaN,6,17,0.3529,12,1,0,5,1,6,3,5,9,0,NaN,0,423,21,3,25,65,0.3800,2,23,0.0900,0,0,0.4100,17,39,88,0.4400,16,45,0.3600,10,23,0.4300,7,10,8,1.3700,0,0,69,0,-0.3000,0,0,0,1,1,52186,2023-08-12 09:00:00-05
1,2,11352250,37,60,A,ALL,4-2-3-1,7.0800,233830000.0000,0.3700,1,16,4,4,14,0,304,NaN,19,30,0.6333,9,4,0,3,1,9,4,12,4,1,NaN,2,206,22,1,21,58,0.3600,5,14,0.3600,0,0,0.5900,8,49,88,0.5600,29,45,0.6400,10,19,0.5300,6,29,11,1.0700,0,0,60,0,0.4000,0,0,0,0,1,52186,2023-08-12 09:00:00-05
2,3,11352250,60,37,H,1ST,NaN,NaN,NaN,0.6600,0,3,1,4,0,0,288,NaN,3,10,0.3000,5,0,0,2,0,1,0,0,3,0,NaN,0,244,7,2,13,31,0.4200,0,7,0.0000,0,0,0.3600,9,18,46,0.3900,6,21,0.2900,3,10,0.3000,3,6,6,0.0800,0,0,35,0,NaN,0,0,0,0,1,52186,2023-08-12 09:00:00-05
3,4,11352250,37,60,A,1ST,NaN,NaN,NaN,0.3400,1,10,2,3,0,0,143,NaN,12,16,0.7500,6,1,0,2,0,7,1,7,3,1,NaN,0,102,15,0,6,23,0.2600,4,10,0.4000,0,0,0.6400,5,28,46,0.6100,15,21,0.7100,6,11,0.5500,2,12,3,0.7400,0,0,28,0,NaN,0,0,0,0,1,52186,2023-08-12 09:00:00-05
4,5,11352250,60,37,H,2ND,NaN,NaN,NaN,0.6000,1,11,0,6,0,0,229,NaN,3,7,0.4286,7,1,0,3,1,5,3,5,6,0,NaN,0,179,14,1,12,34,0.3500,2,16,0.1250,0,0,0.4700,8,21,42,0.5000,10,24,0.4200,7,13,0.5400,4,4,2,1.2900,0,0,34,0,NaN,0,0,0,1,1,52186,2023-08-12 09:00:00-05


In [11]:
player_stats_name = player_stats.merge(players[['player_id', 'name']], on = 'player_id', how = 'left')
player_stats_name.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,ground_duels_total,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal,name
0,1,11352250,84844,60,f,G,1,1900000.0000,6.4000,90,46,0,0,0,27,39,0,6,18,0,0,0,0,0,0,0,0,12,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,-0.2972,0,0,0,0.0000,0.0053,0,0,0,0,0,0,Neto
1,2,11352250,943645,60,f,D,37,11600000.0000,6.9000,75,52,0,0,0,32,36,1,1,1,0,1,0,0,0,0,0,0,6,0,6,6,0,3,0,3,2,0,0,0,1,1,2,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0691,0,0,0,0,0,0,Max Aarons
2,3,11352250,1023567,60,f,D,27,37000000.0000,6.5000,90,102,0,0,0,77,96,0,4,15,0,0,0,0,0,0,0,0,20,0,5,8,4,5,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0122,0,0,0,0,0,0,Illia Zabarnyi
3,4,11352250,830659,60,f,D,25,22000000.0000,6.7000,60,62,0,0,0,43,51,0,3,8,0,0,0,0,0,0,0,0,10,1,3,6,1,2,2,0,2,1,3,1,2,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0172,0,0,0,0,0,0,Marcos Senesi
4,5,11352250,1097425,60,f,D,3,36000000.0000,6.8000,90,75,0,0,0,31,40,1,2,3,1,4,0,0,0,0,0,0,17,1,10,6,4,2,6,3,2,0,3,0,1,0,3,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0493,0,0,0,0,0,0,Miloš Kerkez


In [24]:
filter_home_team = (match_stats['home_away'] == 'H') & (match_stats['period'] == 'ALL')
filter_away_team = (match_stats['home_away'] == 'A') & (match_stats['period'] == 'ALL')
match_stats_all_period = match_stats[match_stats['period'] == 'ALL'].copy()
match_stats_home = match_stats.loc[filter_home_team].copy()
match_stats_away = match_stats.loc[filter_away_team].copy()

df_player_with_own_team_stats = pd.merge(
    player_stats_name,
    match_stats_all_period.add_suffix('_own_team'), 
    left_on=['match_id', 'team_id'],
    right_on=['match_id_own_team', 'team_id_own_team'],
    how='left'
)
#limpiear cols
df_player_with_own_team_stats = df_player_with_own_team_stats.drop(
     columns=['match_id_own_team', 'team_id_own_team'])

match_stats_home_opponent = match_stats_home.add_suffix('_opponent_team')
match_stats_away_opponent = match_stats_away.add_suffix('_opponent_team')

players_whose_team_was_home = df_player_with_own_team_stats[
    df_player_with_own_team_stats['home_away_own_team'] == 'H'
].copy()

players_whose_team_was_away = df_player_with_own_team_stats[
    df_player_with_own_team_stats['home_away_own_team'] == 'A' 
].copy()

merged_for_home_players_team = pd.merge(
    players_whose_team_was_home,
    match_stats_away_opponent, 
    left_on=['match_id', 'opp_team_id_own_team'], 
    right_on=['match_id_opponent_team', 'team_id_opponent_team'], 
    how='left'
)

merged_for_away_players_team = pd.merge(
    players_whose_team_was_away,
    match_stats_home_opponent, 
    left_on=['match_id', 'opp_team_id_own_team'], 
    right_on=['match_id_opponent_team', 'team_id_opponent_team'], 
    how='left'
)

df_final = pd.concat([merged_for_home_players_team, merged_for_away_players_team], ignore_index=True)

df_final.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,ground_duels_total,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal,name,team_match_stat_id_own_team,opp_team_id_own_team,home_away_own_team,period_own_team,formation_own_team,average_team_rating_own_team,total_team_market_value_eur_own_team,possession_percentage_own_team,big_chances_own_team,total_shots_own_team,saves_own_team,corners_own_team,fouls_own_team,passes_successful_own_team,passes_total_own_team,passes_percentage_own_team,tackles_successful_own_team,tackles_total_own_team,tackles_won_percentage_own_team,free_kicks_own_team,yellow_cards_own_team,red_cards_own_team,shots_on_target_own_team,hit_woodwork_own_team,shots_off_target_own_team,blocked_shots_own_team,shots_inside_box_own_team,shots_outside_box_own_team,big_chances_missed_own_team,fouled_final_third_own_team,offsides_own_team,accurate_passes_percentage_own_team,throw_ins_own_team,final_third_entries_own_team,long_balls_successful_own_team,long_balls_total_own_team,long_balls_percentage_own_team,crosses_successful_own_team,crosses_total_own_team,crosses_percentage_own_team,duels_won_successful_own_team,duels_won_total_own_team,duels_won_percentage_own_team,dispossessed_own_team,ground_duels_successful_own_team,ground_duels_total_own_team,ground_duels_percentage_own_team,aerial_duels_successful_own_team,aerial_duels_total_own_team,aerial_duels_percentage_own_team,dribbles_successful_own_team,dribbles_total_own_team,dribbles_percentage_own_team,interceptions_own_team,clearances_own_team,goal_kicks_own_team,expected_goals_own_team,touches_in_penalty_area_own_team,passes_in_final_third_own_team,recoveries_own_team,errors_lead_to_shot_own_team,goals_prevented_own_team,big_saves_own_team,errors_lead_to_goal_own_team,penalty_saves_own_team,big_chances_scored_own_team,round_number_own_team,season_id_own_team,match_datetime_utc_own_team,team_match_stat_id_opponent_team,match_id_opponent_team,team_id_opponent_team,opp_team_id_opponent_team,home_away_opponent_team,period_opponent_team,formation_opponent_team,average_team_rating_opponent_team,total_team_market_value_eur_opponent_team,possession_percentage_opponent_team,big_chances_opponent_team,total_shots_opponent_team,saves_opponent_team,corners_opponent_team,fouls_opponent_team,passes_successful_opponent_team,passes_total_opponent_team,passes_percentage_opponent_team,tackles_successful_opponent_team,tackles_total_opponent_team,tackles_won_percentage_opponent_team,free_kicks_opponent_team,yellow_cards_opponent_team,red_cards_opponent_team,shots_on_target_opponent_team,hit_woodwork_opponent_team,shots_off_target_opponent_team,blocked_shots_opponent_team,shots_inside_box_opponent_team,shots_outside_box_opponent_team,big_chances_missed_opponent_team,fouled_final_third_opponent_team,offsides_opponent_team,accurate_passes_percentage_opponent_team,throw_ins_opponent_team,final_third_entries_opponent_team,long_balls_successful_opponent_team,long_balls_total_opponent_team,long_balls_percentage_opponent_team,crosses_successful_opponent_team,crosses_total_opponent_team,crosses_percentage_opponent_team,duels_won_successful_opponent_team,duels_won_total_opponent_team,duels_won_percentage_opponent

# Estudio Defensas

In [28]:
centrales = ['Virgil van Dijk', 'Antonio Rüdiger', 'Sergio Ramos']
laterales = ['Trent Alexander-Arnold', 'Daniel Carvajal', 'Kyle Walker']
medio_campo = ['Kevin De Bruyne', 'Luka Modrić', 'Toni Kroos', 'Pedri']
delanteros = ['Mohamed Salah', 'Lionel Messi', 'Karim Benzema', 'Erling Haaland']

top_players = centrales + laterales + medio_campo + delanteros

filtro_top_players = df_final['name'].isin(top_players)
df_final_jugadores_top = df_final.loc[filtro_top_players]
df_final_jugadores_top.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,ground_duels_total,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal,name,team_match_stat_id_own_team,opp_team_id_own_team,home_away_own_team,period_own_team,formation_own_team,average_team_rating_own_team,total_team_market_value_eur_own_team,possession_percentage_own_team,big_chances_own_team,total_shots_own_team,saves_own_team,corners_own_team,fouls_own_team,passes_successful_own_team,passes_total_own_team,passes_percentage_own_team,tackles_successful_own_team,tackles_total_own_team,tackles_won_percentage_own_team,free_kicks_own_team,yellow_cards_own_team,red_cards_own_team,shots_on_target_own_team,hit_woodwork_own_team,shots_off_target_own_team,blocked_shots_own_team,shots_inside_box_own_team,shots_outside_box_own_team,big_chances_missed_own_team,fouled_final_third_own_team,offsides_own_team,accurate_passes_percentage_own_team,throw_ins_own_team,final_third_entries_own_team,long_balls_successful_own_team,long_balls_total_own_team,long_balls_percentage_own_team,crosses_successful_own_team,crosses_total_own_team,crosses_percentage_own_team,duels_won_successful_own_team,duels_won_total_own_team,duels_won_percentage_own_team,dispossessed_own_team,ground_duels_successful_own_team,ground_duels_total_own_team,ground_duels_percentage_own_team,aerial_duels_successful_own_team,aerial_duels_total_own_team,aerial_duels_percentage_own_team,dribbles_successful_own_team,dribbles_total_own_team,dribbles_percentage_own_team,interceptions_own_team,clearances_own_team,goal_kicks_own_team,expected_goals_own_team,touches_in_penalty_area_own_team,passes_in_final_third_own_team,recoveries_own_team,errors_lead_to_shot_own_team,goals_prevented_own_team,big_saves_own_team,errors_lead_to_goal_own_team,penalty_saves_own_team,big_chances_scored_own_team,round_number_own_team,season_id_own_team,match_datetime_utc_own_team,team_match_stat_id_opponent_team,match_id_opponent_team,team_id_opponent_team,opp_team_id_opponent_team,home_away_opponent_team,period_opponent_team,formation_opponent_team,average_team_rating_opponent_team,total_team_market_value_eur_opponent_team,possession_percentage_opponent_team,big_chances_opponent_team,total_shots_opponent_team,saves_opponent_team,corners_opponent_team,fouls_opponent_team,passes_successful_opponent_team,passes_total_opponent_team,passes_percentage_opponent_team,tackles_successful_opponent_team,tackles_total_opponent_team,tackles_won_percentage_opponent_team,free_kicks_opponent_team,yellow_cards_opponent_team,red_cards_opponent_team,shots_on_target_opponent_team,hit_woodwork_opponent_team,shots_off_target_opponent_team,blocked_shots_opponent_team,shots_inside_box_opponent_team,shots_outside_box_opponent_team,big_chances_missed_opponent_team,fouled_final_third_opponent_team,offsides_opponent_team,accurate_passes_percentage_opponent_team,throw_ins_opponent_team,final_third_entries_opponent_team,long_balls_successful_opponent_team,long_balls_total_opponent_team,long_balls_percentage_opponent_team,crosses_successful_opponent_team,crosses_total_opponent_team,crosses_percentage_opponent_team,duels_won_successful_opponent_team,duels_won_total_opponent_team,duels_won_percentage_opponent

In [41]:
df_centrales = df_final_jugadores_top.loc[(df_final_jugadores_top['name'].isin(centrales)) & (df_final_jugadores_top['minutes_played'] >= 45)]
metricas_clave_centrales = ['aerials_lost', 'aerials_won', 'aerial_duels_successful_own_team',
                        'big_chances_opponent_team', 'big_chances_scored_opponent_team', 'goals',
                        'clearances', 'clearances_own_team', 'corners_opponent_team',
                        'crosses_successful_opponent_team', 'crosses_total_opponent_team',
                        'dribbled_past', 'duels_lost', 'duels_won', 'errors_lead_to_goal_own_team',
                        'errors_lead_to_shot_own_team', 'errors_leading_to_goal', 'errors_leading_to_shot',
                        'expected_goals_opponent_team', 'expected_goals_own_team',
                        'final_third_entries_opponent_team', 'formation_own_team', 'fouls_committed',
                        'fouls_opponent_team', 'ground_duels_total', 'ground_duels_won', 'ground_duels_percentage_own_team',
                        'interceptions', 'interceptions_own_team',
                        'long_balls_accurate', 'long_balls_total', 'minutes_played', 'offsides_opponent_team',
                        'own_goals', 'passes_accurate', 'passes_total', 'passes_total_own_team',
                        'penalty_committed', 'possession_lost',
                        'possession_percentage_own_team', 'recoveries_own_team',
                        'shots_blocked_by_player', 'shots_on_target_opponent_team', 'shots_on_target_own_team',
                        'sofascore_rating', 'tackles', 'tackles_won_percentage_own_team', 'total_shots_opponent_team', 'total_shots_own_team',
                        'touches_in_penalty_area_opponent_team']


df_centrales['aerials_won_vs_team_ratio'] = df_centrales['aerials_won'] / df_centrales['aerial_duels_successful_own_team']
df_centrales['ground_duels_won_vs_team_ratio'] = df_centrales['ground_duels_won'] / df_centrales['ground_duels_percentage_own_team']
df_centrales['interceptions_vs_team_ratio'] = df_centrales['interceptions'] / df_centrales['interceptions_own_team']
df_centrales['clearances_vs_team_ratio'] = df_centrales['clearances'] / df_centrales['clearances_own_team']
df_centrales['tackles_vs_team_ratio'] = df_centrales['tackles'] / df_centrales['tackles_won_percentage_own_team']
df_centrales['passes_total_vs_team_ratio'] = df_centrales['passes_total'] / df_centrales['passes_total_own_team']

nuevas = ['aerials_won_vs_team_ratio', 'ground_duels_won_vs_team_ratio', 'interceptions_vs_team_ratio', 'clearances_vs_team_ratio',
          'tackles_vs_team_ratio', 'passes_total_vs_team_ratio']
metricas_clave_centrales = metricas_clave_centrales + nuevas
df_centrales_means = df_centrales.groupby(['player_id', 'name'])[metricas_clave_centrales].agg('mean', 'count').reset_index()
df_centrales_means = df_centrales_means.sort_values(by = 'sofascore_rating', ascending=False).head()
df_centrales_means.head()

C:\Users\javie\AppData\Local\Temp\ipykernel_37636\2855617770.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_centrales['aerials_won_vs_team_ratio'] = df_centrales['aerials_won'] / df_centrales['aerial_duels_successful_own_team']
C:\Users\javie\AppData\Local\Temp\ipykernel_37636\2855617770.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_centrales['ground_duels_won_vs_team_ratio'] = df_centrales['ground_duels_won'] / df_centrales['ground_duels_percentage_own_team']
C:\Users\javie\AppData\Local

,player_id,name,aerials_lost,aerials_won,aerial_duels_successful_own_team,big_chances_opponent_team,big_chances_scored_opponent_team,goals,clearances,clearances_own_team,corners_opponent_team,crosses_successful_opponent_team,crosses_total_opponent_team,dribbled_past,duels_lost,duels_won,errors_lead_to_goal_own_team,errors_lead_to_shot_own_team,errors_leading_to_goal,errors_leading_to_shot,expected_goals_opponent_team,expected_goals_own_team,final_third_entries_opponent_team,fouls_committed,fouls_opponent_team,ground_duels_total,ground_duels_won,ground_duels_percentage_own_team,interceptions,interceptions_own_team,long_balls_accurate,long_balls_total,minutes_played,offsides_opponent_team,own_goals,passes_accurate,passes_total,passes_total_own_team,penalty_committed,possession_lost,possession_percentage_own_team,recoveries_own_team,shots_blocked_by_player,shots_on_target_opponent_team,shots_on_target_own_team,sofascore_rating,tackles,tackles_won_percentage_own_team,total_shots_opponent_team,total_shots_own_team,touches_in_penalty_area_opponent_team,aerials_won_vs_team_ratio,ground_duels_won_vs_team_ratio,interceptions_vs_team_ratio,clearances_vs_team_ratio,tackles_vs_team_ratio,passes_total_vs_team_ratio
0,10710,Sergio Ramos,1.3721,2.5581,14.4651,2.4651,0.7674,0.1163,4.5349,17.0233,5.0465,4.1628,17.9302,0.3953,2.8372,4.8372,0.0000,0.0000,0.0000,0.0233,1.4614,1.1932,2.0233,1.0233,11.2093,0.3721,1.9535,0.5272,1.3953,9.1860,6.7674,9.5581,88.2326,1.9535,0.0233,55.0000,60.8837,502.5116,0.0465,6.7209,0.5370,36.2558,0.8140,4.3721,4.3256,7.3047,1.3953,0.6703,11.9767,12.8837,1.3256,0.1729,3.7434,0.1598,0.2688,2.1749,0.1200
2,151545,Virgil van Dijk,1.1014,3.4493,13.8696,2.0000,0.6304,0.0797,4.0507,15.1594,3.5725,2.9348,12.8768,0.1957,1.8478,4.6812,0.0652,0.1377,0.0290,0.0290,1.1321,2.1943,1.7174,0.4420,9.3261,0.1377,1.1667,0.4685,1.1449,8.2899,4.6087,7.8116,89.6739,2.7754,0.0000,72.3406,79.6957,586.7826,0.0072,7.7536,0.6133,28.7536,0.5580,3.5435,6.6159,7.2587,0.8623,0.6031,9.3261,18.3623,5.4203,0.2513,2.5132,0.1388,0.2755,1.4888,0.1354
1,142622,Antonio Rüdiger,0.8134,1.6045,11.0970,1.3955,0.5746,0.0448,2.7687,13.7687,3.9179,3.2537,14.8358,0.2388,2.0149,3.1343,0.0224,0.0522,0.0075,0.0075,1.0107,1.8763,1.5970,0.7836,12.8731,0.2239,1.1866,0.5240,0.6194,8.4851,3.6343,6.8284,89.1716,1.2985,0.0075,61.1866,67.8134,615.2463,0.0075,7.5597,0.6152,20.7090,0.4851,3.1045,5.9328,7.0537,1.0448,0.6185,9.6269,15.9328,3.8134,0.1525,2.2531,0.0766,0.2045,1.7748,0.1099
